# Projeto: Previsão de Custos de Seguro Médico

**Autor:** Diego de Paula Silva
**Data:** 05/11/2025

Este notebook tem como objetivo prever os custos de seguro usando dados demográficos e de saúde. Vamos comparar 4 modelos de regressão.

## Carregamento e preparação dos dados
Importamos as bibliotecas e carregamos os dados. As features categorias (sex, smoker, region) serão convertidas em números usando One-Hot Encoding (pd.get_dummies) para que o modelo possa processa-las.

In [33]:
import pandas as pd 

dados = pd.read_csv("insurance.csv")

dados_processados = dados.copy()
dados_processados = pd.get_dummies(dados_processados, drop_first=True)

In [34]:
dados_processados.head()

,age,bmi,children,charges,sex_male,smoker_yes,region_northwest,region_southeast,region_southwest
0,19,27.900,0,16884.92400,False,True,False,False,True
1,18,33.770,1,1725.55230,True,False,False,True,False
2,28,33.000,3,4449.46200,True,False,False,True,False
3,33,22.705,0,21984.47061,True,False,True,False,False
4,32,28.880,0,3866.85520,True,False,True,False,False


In [35]:
dados_processados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   age               1338 non-null   int64  
 1   bmi               1338 non-null   float64
 2   children          1338 non-null   int64  
 3   charges           1338 non-null   float64
 4   sex_male          1338 non-null   bool   
 5   smoker_yes        1338 non-null   bool   
 6   region_northwest  1338 non-null   bool   
 7   region_southeast  1338 non-null   bool   
 8   region_southwest  1338 non-null   bool   
dtypes: bool(5), float64(2), int64(2)
memory usage: 48.5 KB


In [36]:
dados_processados.describe()

,age,bmi,children,charges
count,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918,13270.422265
std,14.049960,6.098187,1.205493,12110.011237
min,18.000000,15.960000,0.000000,1121.873900
25%,27.000000,26.296250,0.000000,4740.287150
50%,39.000000,30.400000,1.000000,9382.033000
75%,51.000000,34.693750,2.000000,16639.912515
max,64.000000,53.130000,5.000000,63770.428010


## Definição das Features (X) e Alvo (Y)
O alvo (y) é a coluna 'charges'. As features (X) são todas as outras colunas relevantes

In [37]:
features = ['age', 'bmi', 'children', 'sex_male', 'smoker_yes', 'region_northwest', 'region_southeast', 'region_southwest']
X = dados_processados[features]
y = dados_processados['charges']

## Divisão dos dados (Treino e Teste)
Dividimos os dados em 75% treino e 25% para teste. Usamos `random_state=1` para garantir que nossos resultados sejam reprodutíveis.

In [38]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

## 4. Avaliação dos Modelos

Vamos agora treinar e avaliar nossos 4 modelos para encontrar o que tem o menor Erro Absoluto Médio (MAE).

### Modelo 1: Árvore de Decisão Simples

In [39]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
modelo = DecisionTreeRegressor(random_state=1)
modelo.fit(X_train, y_train)
preds = modelo.predict(X_test)
mae_dt_simples =  mean_absolute_error(y_test, preds)
print("MAE da árvore de decisão:", mae_dt_simples)

MAE da árvore de decisão: 3334.3237347761196


### Modelo 2: Árvore de Decisão Otimizada (max_leaf_nodes)

In [40]:
#arvore de decisão otimizada
#usamos uma função para calcular o MAE para diferentes tamanhos de árvore, afim de decidir qual tamanho terá o melhor desempenho
def get_mae(max_leaf_nodes, X_train, X_test, y_train, y_test):
    modelo = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=1)
    modelo.fit(X_train, y_train)
    preds_val = modelo.predict(X_test)
    mae = mean_absolute_error(y_test, preds_val)
    return(mae)

candidatos_tamanho_arvore = [10, 25, 50, 100, 250, 500]

pontuacoes = {nodes: get_mae(nodes, X_train, X_test, y_train, y_test) for nodes in candidatos_tamanho_arvore}

melhor_tam_arvore = min(pontuacoes, key=pontuacoes.get)
mae_dt_otimizado = pontuacoes[melhor_tam_arvore]
print("Melhor tamanho da árvore:", melhor_tam_arvore)
print("MAE:", mae_dt_otimizado)

Melhor tamanho da árvore: 25
MAE: 2581.0067603456937


### Modelo 3: Random Forest Simples (Padrão)

In [41]:
#floresta aleatória
from sklearn.ensemble import RandomForestRegressor
modelo_floresta = RandomForestRegressor(random_state=1)
modelo_floresta.fit(X_train, y_train)
preds = modelo_floresta.predict(X_test)
mae_rf_simples = mean_absolute_error(y_test, preds)
print("MAE da floresta aleatória:", mae_rf_simples)

MAE da floresta aleatória: 2755.237458776299


### Modelo 4: Random Forest Otimizado (max_leaf_nodes)

In [42]:
#floresta aleatória otimizada
#função para testar diferentes tamanhos de árvores na floresta aleatória
def get_mae_forest(max_leaf_nodes, X_train, X_test, y_train, y_test):
    modelo = RandomForestRegressor(max_leaf_nodes=max_leaf_nodes, random_state=1)
    modelo.fit(X_train, y_train)
    preds_val = modelo.predict(X_test)
    mae = mean_absolute_error(y_test, preds_val)
    return(mae)

propostas_max_leaf_nodes = [10, 25, 50, 100, 250, 500] 
pontuacoes_forest = {n: get_mae_forest(n, X_train, X_test, y_train, y_test) for n in propostas_max_leaf_nodes} 

melhor_max_leaf_nodes = min(pontuacoes_forest, key=pontuacoes_forest.get)
mae_rf_otimizado = pontuacoes_forest[melhor_max_leaf_nodes]
print("Melhor tamanho da arvore:", melhor_max_leaf_nodes)
print("MAE:", mae_rf_otimizado)

Melhor tamanho da arvore: 25
MAE: 2505.744814210039


## Conclusão e comparação de resultados 
Vamos reunir todos os resultados de MAE em uma tabela para comparação direta.

In [ ]:
#Dicionario com os valores de MAE para comparação
mae_valores = {"Arvore de Decisão Simples": mae_dt_simples,
               "Arvore de Decisão Otimizada": mae_dt_otimizado,
               "Floresta Aleatória Simples": mae_rf_simples,
               "Floresta Aleatória Otimizada": mae_rf_otimizado
               }

resultados = pd.DataFrame(list(mae_valores.items()), columns=['Modelo', 'Mae'])

print(resultados)


                         Modelo          Mae
0     Arvore de Decisão Simples  3334.323735
1   Arvore de Decisão Otimizada  2581.006760
2    Floresta Aleatória Simples  2755.237459
3  Floresta Aleatória Otimizada  2505.744814
